<a href="https://colab.research.google.com/github/mowSokol/test_Sokolov/blob/main/%D0%94%D0%BE%D0%B1%D1%80%D0%BE_%D0%BF%D0%BE%D0%B6%D0%B0%D0%BB%D0%BE%D0%B2%D0%B0%D1%82%D1%8C_%D0%B2_Colaboratory!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip --version

pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


Установка PySpark

In [4]:
!pip install pyspark py4j

PySpark стоит

In [5]:
from pyspark import SparkContext, SparkConf



# Настройка Spark

conf = SparkConf().setAppName("Simple RDD Example").setMaster("local[*]")

sc = SparkContext(conf=conf)



# 1. Создание RDD из списка чисел

numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

rdd = sc.parallelize(numbers)



# 2. Трансформации: Фильтрация чётных чисел

even_numbers_rdd = rdd.filter(lambda x: x % 2 == 0)



# 3. Действие: Подсчёт суммы чётных чисел

sum_even_numbers = even_numbers_rdd.sum()



# Вывод результата

print("Чётные числа:", even_numbers_rdd.collect())

print("Сумма чётных чисел:", sum_even_numbers)



a = 5 + 6

print(a)

# Остановка SparkContext

sc.stop()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Read CSV Example, master=local[*]) created by getOrCreate at <ipython-input-1-fea8a025f6cf>:4 

Подтверждаем что PySpark стоит

In [6]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.6 MB/s eta 0:00:00


In [7]:
import csv
from faker import Faker
import random

fake = Faker()

num_records = 100000

http_methods = ['GET', 'POST', 'PUT', 'DELETE']
response_codes = [200, 301, 404, 500]

file_path = "web_server_logs.csv"

with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ip', 'timestamp', 'method', 'url', 'response_code', 'response_size'])

    for _ in range(num_records):
        ip = fake.ipv4()
        timestamp = fake.date_time_this_year().isoformat()
        method = random.choice(http_methods)
        url = fake.uri_path()
        response_code = random.choice(response_codes)
        response_size = random.randint(100, 10000)

        writer.writerow([ip, timestamp, method, url, response_code, response_size])

print(f"Сгенерировано {num_records} записей и сохранено в {file_path}")

Сгенерировано 100000 записей и сохранено в web_server_logs.csv


Сгенерировали записи и получили файл


**ИТОГОВОЕ ЗАДАНИЕ по PySpark**

In [56]:
from pyspark.sql import SparkSession

# Создание SparkSession
spark = SparkSession.builder.appName("Read CSV Example").getOrCreate()

# Чтение CSV-файла
df = spark.read.csv("/content/web_server_logs.csv", header=True, inferSchema=True)

# Печать схемы DataFrame
df.printSchema()

df.createOrReplaceTempView("df")

print("Top 10 active IP adress:")
top_ip = spark.sql("SELECT ip, COUNT(ip) AS request_count FROM df GROUP BY ip").limit(10).show()

print("Request count by HTTP method:")
method = spark.sql("SELECT method, COUNT(method) AS method_count FROM df GROUP BY method").show()

count_code = spark.sql("SELECT COUNT(response_code) AS count_response_code FROM df WHERE response_code = 404").show()
#print("number of 404 response codes:", count_code)

print("total response size by day:")
response_size = spark.sql("SELECT date, SUM(response_size) as total_response_size FROM (SELECT to_date(timestamp, 'yyyy-MM-dd') as date, response_size FROM df) AS T GROUP BY date ORDER BY date").show()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: integer (nullable = true)
 |-- response_size: integer (nullable = true)

Top 10 active IP adress:
+--------------+-------------+
|            ip|request_count|
+--------------+-------------+
|  39.43.102.57|            1|
| 114.44.72.220|            1|
|200.166.61.168|            1|
| 46.95.234.188|            1|
|95.173.211.205|            1|
|158.113.136.77|            1|
| 58.163.71.113|            1|
|  4.109.129.72|            1|
| 218.120.8.111|            1|
|  8.187.55.222|            1|
+--------------+-------------+

Request count by HTTP method:
+------+------------+
|method|method_count|
+------+------------+
|  POST|       25257|
|DELETE|       25005|
|   PUT|       24765|
|   GET|       24973|
+------+------------+

+-------------------+
|count_response_code|
+-------------------+
|              250